In [48]:
from transformers import CLIPSegProcessor, CLIPSegForImageSegmentation
import torch
import matplotlib.pyplot as plt
import cv2
import time
import numpy as np
from transformers import CLIPTokenizer

In [43]:
num_classes = 7
processor = CLIPSegProcessor.from_pretrained("./clipseg/clipseg-rd64-refined")
model = CLIPSegForImageSegmentation.from_pretrained("./clipseg/clipseg-rd64-refined")

In [72]:
t0 = time.time()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# tokenizer = CLIPTokenizer.from_pretrained("./clipseg/clipseg-rd64-refined")
prompts = ["background", "red", "orange", "yellow", "green", "blue", "indigo", "violet", "human", "people"]
# prompts = tokenizer(prompts, padding=True, return_tensors="pt")
# if device=="cuda":
#     prompts = prompts.cuda()

image = cv2.imread("./clipseg/0.png")
# from torchvision import transforms
# to_tensor = transforms.ToTensor()
# image = to_tensor(image)

detected_thold = 50

orig_size = image.shape[:2][::-1]

inputs = processor(text=prompts, images=[image] * len(prompts), padding="max_length", return_tensors="pt")

inputs.to(device)

with torch.no_grad():
    outputs = model(**inputs)

t1 = time.time()
print(t1 - t0)
inputs

1.6074435710906982


{'input_ids': tensor([[49406,  5994, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407],
        [49406,   736, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 

In [45]:
preds = outputs.logits # shape=[10, 352,352] tensor
segm = preds.cpu().detach().numpy().transpose(1, 2, 0)
segm = cv2.resize(segm, orig_size, interpolation=cv2.INTER_CUBIC)
segm = segm.argmax(axis=2).astype(np.uint8)

In [46]:
segm

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [47]:
class_ids = []
labels = []
masks = np.zeros((segm.shape[0], segm.shape[1], num_classes))
count = 0
for class_id in range(1, num_classes+1):
    temp = np.zeros((segm.shape[0], segm.shape[1]))
    temp[segm == class_id] = 1
    if np.sum(temp) > detected_thold:
        masks[:, :, count] = temp
        class_ids.append(class_id)
        labels.append(prompts[class_id])
        count += 1
res = {'class_ids': class_ids, 'labels': labels, 'masks': masks}
res


{'class_ids': [1, 2, 4, 5, 6, 7],
 'labels': ['red', 'orange', 'green', 'blue', 'indigo', 'violet'],
 'masks': array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..